In [ ]:
"""
    1.抓secret_key裡面的資料（重要變數）
    2.啟用API伺服器基本樣板，啟用伺服器基本
    3.製作一個測試用接口
    4.編寫用戶追蹤後的行為
    5.製作有圖案以及文字的回覆
    6.針對message event的設定
    7.啟動server

"""

In [ ]:
"""

    1.抓secret_key裡面的資料
    
"""
# 載入json處理套件
import json

# 載入基礎設定檔，本機執行所以路徑可以相對位置，在test.py要改成絕對路徑，因為是使用kernel開啟
secretFile=json.load(open("./secret_key",'r'))

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

#讓requests能準確的傳送包，這邊ip要特別注意
ip_location='10.1.1.16'

# channel_access_token是用於傳送封包去給line的認證使用類似這個是私鑰，而公鑰已經丟到line那邊，拿著這個就可以有VIP特權
line_bot_api = LineBotApi(secretFile.get("channel_access_token"))
# secret_key是用於當line傳送封包過來時確定line是否為本尊，沒有被仿冒
handler = WebhookHandler(secretFile.get("secret_key"))
# rich_menu_id用於將所指定的rich menu綁定到加好友的用戶上
menu_id = secretFile.get("rich_menu_id")
# server_url是用於讓Line官方找得到你
server_url = secretFile.get("server_url")


In [ ]:
"""

  2.啟用伺服器基本樣板，啟用伺服器基本 

"""

# 引用Web Server套件
from flask import Flask, request, abort , jsonify

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


# 設定Server啟用細節，這邊使用相對位置，test.py中使用絕對位置，因為是使用kernel開啟
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # Line 封包的加密機制
    signature = request.headers['X-Line-Signature']

    # 讀取封包內資料
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # 解密的動作
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""
    3.製作一個測試用接口
"""
@app.route('/hello',methods=['GET'])
def hello():
    return 'Hello World!!'

In [ ]:
"""

    4.撰寫用戶關注事件發生時的動作
            1. 取得用戶個資，並存回伺服器
            2. 回應用戶，歡迎用的文字消息與圖片消息
            
"""

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊做成合適Json
    user_info = {  
        "user_open_id":user_profile.user_id,
        "user_nick_name":user_profile.display_name,
        #status留給學弟做應用
        "user_status" : "",
        "user_img" : user_profile.picture_url,
        "user_register_menu" : menu_id
    }
    # 將json傳回API Server
    Endpoint='http://%s:5001/users' % (ip_location)
    # header要特別註明是json格式
    Header={'Content-Type':'application/json'}
    # 傳送post對API server新增資料 
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(user_info))
      
    
    # 針對剛加入的用戶回覆文字消息與圖片消息
    line_bot_api.reply_message(
         event.reply_token,
         reply_message_list
    )

In [ ]:
"""

    5.製作有圖案以及文字的回覆

"""


reply_message_list = [
        ImageSendMessage(original_content_url='https://%s/images/certificate.jpg' %server_url,
                         preview_image_url='https://%s/images/certificate.jpg' %server_url),
        TextSendMessage(text="歡迎使用考古題聊天機器人"),
        TextSendMessage(text="今天天氣真好")
    ]



In [ ]:
"""

    6.針對message event的設定
    當用戶發出文字消息時，判斷文字內容是否包含一些關鍵，
    若有，則回傳客製化訊息
    若無，則回傳預設訊息。

"""

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
#將這次event的參數抓進來
def handle_message(event):
    user_profile = event.source.user_id
    # 由於在quick_reply的地方，在按鈕給了text針對按鈕的text不予回應
    if (event.message.text.find('ignore me')!= -1):
        pass
    # 當用戶輸入detail時判斷成立
    elif (event.message.text.find(':')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    # 收到不認識的訊息時，所給予的回覆    
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="聊天機器人好棒棒"))



In [ ]:

'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    ImageSendMessage
)

# 圖片消息的基本建構特徵
image_message = ImageSendMessage(
    original_content_url='https://%s/images/newton1.jpg' % server_url,
    preview_image_url='https://%s/images/newton1.jpg' % server_url
)
image_message2 = ImageSendMessage(
    original_content_url='https://%s/images/southwind1.jpg' % server_url,
    preview_image_url='https://%s/images/southwind1.jpg' % server_url
)

'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    ":牛頓":image_message,
    ":南風":image_message2,
    ":登入訊息":reply_message_list
}



In [ ]:
'''
    7.啟動server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)